In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

dfAuthors = pd.read_csv('DBPediaAuthorsReducedSample.csv', sep=',', low_memory=False)
dfAuthorDnB = pd.DataFrame()
#dfAuthor = pd.read_csv('DnbDataAuthorsNew.csv', sep=',', low_memory=False)
display(dfAuthors.head())

,URI,birthDate,deathDate,gender,label,links,work
0,URI,http://dbpedia.org/ontology/birthDate,http://dbpedia.org/ontology/work,http://xmlns.com/foaf/0.1/gender,http://www.w3.org/2000/01/rdf-schema#label,www.w3.org/2002/07/owl#sameAs,http://dbpedia.org/ontology/work
1,URI,XMLSchema#date,XMLSchema#list,XMLSchema#string,XMLSchema#string,XMLSchema#list,XMLSchema#list
2,http://www.w3.org/2002/07/owl#Thing,http://www.w3.org/2001/XMLSchema#date,https://www.w3.org/2001/XMLSchema#list,http://www.w3.org/2001/XMLSchema#string,http://www.w3.org/2001/XMLSchema#string,https://www.w3.org/2001/XMLSchema#list,https://www.w3.org/2001/XMLSchema#list
3,http://dbpedia.org/resource/Abbie_Hoffman,1936-11-30,1989-04-12,male,Abbie Hoffman,\thttp://viaf.org/viaf/79408884\thttp://www.wi...,\tSteal_This_Book\tWoodstock_Nation_(book)
4,http://dbpedia.org/resource/Adam_Oehlenschläger,1779-11-14,1850-01-20,male,Adam Oehlenschläger,\thttp://dbpedia.org/resource/Adam_Oehlenschlä...,NaN


In [33]:
import rdflib
from rdflib import URIRef
import pandas as pd
import csv
from urllib.error import HTTPError


def convertResultToDataFrame( identifier ):                        
        g = rdflib.Graph()
        g.load(''.join([identifier, '/about/rdf']))
                             
        author = {}
        author['uri'] = identifier
        
        searchedFeatures = ['gender', 'preferredNameEntityForThePerson', 'dateOfBirth', 'dateOfDeath', 'preferredNameForThePerson']
        skipFeatures = []
                    
        
        for p,o in g.predicate_objects(URIRef(author['uri'])):
            pLiteral = p.split('#')[-1]
            if pLiteral in searchedFeatures:
                if pLiteral == 'gender':
                    author[pLiteral] = o.split('#')[-1] 
                
                if type(o) is rdflib.term.Literal:
                    object1 = o.split('#')[-1]
                    if(pLiteral in author):
                        author[pLiteral] = '\t '.join([author[pLiteral], object1])
                    else:
                        author[pLiteral] = object1
                
                if 'preferredNameEntityForThePerson' in pLiteral:
                    for pEntity, oEntity in g.predicate_objects(o):
                        author[pEntity.split('#')[-1]] = oEntity.split('#')[-1]
                else:
                    author[pLiteral] = o.split('#')[-1].replace('\n', '')
        return pd.DataFrame(author, index=[0])

dfAuthorDnB = pd.DataFrame()
counter = 0
for row in dfAuthors['links']:
    for link in row.split('\t'):
        if 'http://d-nb.info/gnd/' in link:
            #if link not in dfAuthor['uri'].values:
                dfAuthorDnB = dfAuthorDnB.append(convertResultToDataFrame(link), ignore_index=True, sort=True)
                if counter == 10:
                    dfAuthorDnB.to_csv('DnbDataAuthorsReduced.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
                    counter = 0
                else:
                    counter = counter + 1

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(dfAuthorDnB)
#dfAuthorDnB.to_csv('DnbDataAuthorsNew.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)

,dateOfBirth,dateOfDeath,forename,gender,nameAddition,personalName,preferredNameForThePerson,surname,uri
0,1936-01-01,1989-01-01,Abbie,notKnown,NaN,NaN,"Hoffman, Abbie",Hoffman,http://d-nb.info/gnd/119047438
1,1779-11-14,1850-01-20,Adam Gottlob,male,NaN,NaN,"Oehlenschläger, Adam Gottlob",Oehlenschläger,http://d-nb.info/gnd/119071606
2,1937-01-01,1995-08-23,Adam,notKnown,NaN,NaN,"Wiśniewski-Snerg, Adam",Wiśniewski-Snerg,http://d-nb.info/gnd/1028840020
3,1929-05-16,2012-03-27,Adrienne,female,NaN,NaN,"Rich, Adrienne",Rich,http://d-nb.info/gnd/118744895
4,1785-03-07,1873-05-22,Alessandro,male,NaN,NaN,"Manzoni, Alessandro",Manzoni,http://d-nb.info/gnd/118577409
5,1913-01-01,1987-01-01,Alfred,notKnown,NaN,NaN,"Bester, Alfred",Bester,http://d-nb.info/gnd/118658654
6,1944-02-09,NaN,Alice,female,NaN,NaN,"Walker, Alice",Walker,http://d-nb.info/gnd/118884433
7,1952-01-01,NaN,Amy,female,NaN,NaN,"Tan, Amy",Tan,http://d-nb.info/gnd/119363925
8,1621-01-01,1678-01-01,Andrew,notKnown,NaN,NaN,"Marvell, Andrew",Marvell,http://d-nb.info/gnd/118578448
9,1928-01-01,2012-01-01,Andrew,notKnown,NaN,NaN,"Sarris, Andrew",Sarris,http://d-nb.info/gnd/122951786


In [ ]:
#Add works
import certifi
import urllib3
import urllib
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    works = []
    record = False
    
    def handle_starttag(self, tag, attrs):
        if 'a' in tag:
            self.record = True

    def handle_endtag(self, tag):
        if 'a' in tag:
            self.record = False

    def handle_data(self, data):
        if self.record:
            self.works.append(data.replace('\t', '').replace('\n', ''))

def retrieveListWork(gndIdentifier, index):
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    if index < 10:
        url = "https://portal.dnb.de/opac.htm?method=simpleSearch&reset=true&cqlMode=true&query=auRef%3D{}&selectedCategory=any".format(gndIdentifier)
    else:
        url = "https://portal.dnb.de/opac.htm?method=simpleSearch&query=auRef%3D{}&sortOrderIndex=&cqlMode=true&hitnumber={}".format(gndIdentifier, index)
    r = http.request('GET', url)
    parser = MyHTMLParser()
    parser.works = []
    htmlPage = r.data.decode('utf-8')
    if '<table id="searchresult" border="0" cellpadding="0" cellspacing="0" class="searchresult" summary="Suchergebnis">' in htmlPage:
        htmlPage = htmlPage.split('<table id="searchresult" border="0" cellpadding="0" cellspacing="0" class="searchresult" summary="Suchergebnis">')[-1]
        htmlPage = htmlPage.split('</table>')[0]
        htmlPage = htmlPage.replace('\\t', '')
        htmlPage = htmlPage.replace('\\n', '')
        htmlPage = htmlPage.replace('<br/>', '|')
        parser.feed(htmlPage)
    
    return parser.works

dfAuthorWithWorks = pd.DataFrame()

for index, row in dfAuthor.iterrows():
    index = 1
    author_works = []
    newResults = True
    while newResults:
        newWorks = retrieveListWork(row['gndIdentifier'], index)
        if len(newWorks) > 0:
            author_works.append(newWorks)
            index += 10
        else:
            row['work'] = author_works
            dfAuthorWithWorks = dfAuthorWithWorks.append(row)
            newResults = False
        
display(dfAuthorWithWorks)

In [ ]:
# Double-Check if hitnumber plus 10 can be fixed!
import csv
dfAuthorWithWorks.to_csv('DnbDataAuthorsWorkNew.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)